# Minimum working example for modeling photostimulation in Zebrafish

Data Loading

In [ ]:
import numpy as np
import adaptive_latents.stimulation.photostimulation as ps
import adaptive_latents as al
from adaptive_latents import NumpyTimedDataSource, Bubblewrap

In [ ]:
d = al.datasets.Naumann24uDataset(al.datasets.Naumann24uDataset.sub_datasets[0])
bp = d.dataset_base_path / d.sub_dataset

# Load the files
stim = np.loadtxt(bp / 'stimmed.txt') 
"""1st entry: frame number,
2nd entry: ignore,
3rd entry: angle of motion L,
4th entry: angle of motion R,
5th entry: timestamp,"""
C = np.loadtxt(bp / 'analysis_proc_C.txt')#
"""Calcium imaging. 
1st entry is neuron ID,
2nd is time (frame)"""
photostim = np.load(bp / 'photostims.npy')
"""1st entry: frame number,
2nd entry: counter of stims,
3rd entry: neuron ID,
4th entry: position X of neuron,
5th entry: position Y of neuron,"""

# extra note: Fs= 2.3 Hz

Run BW

In [ ]:
#Perform dimensionality reduction
p = al.Pipeline([
    al.CenteringTransformer(),
    al.proSVD(k=7),
])
smallC_photo = p.offline_run_on(C.T)


In [ ]:
bw = Bubblewrap(M=100, num=1000, B_thresh=-10, num_grad_q=1, log_level=2)

bw.offline_run_on(smallC_photo)

Extract the log predictive probability values

Create a data frame for photostimulation and log PP rankings

In [ ]:
predictions = np.array([bw.log['log_pred_p']])
final_df=ps.rank_neurons(stim, C, photostim, predictions)

In [ ]:
#let's sort by change in log PP ranking
df_sorted = final_df.sort_values(by=['ranking', 'frame_number'])

df_sorted.iloc[:15]

Plotting trajectories with PCA and proSVD for particular neuron trials

In [ ]:
ps.plot_photostim(31,'pca',C, photostim, final_df)

In [ ]:
ps.plot_photostim(31,'proSVD',C, photostim, final_df)

Plot the Log PP of a determined neuron 

Note: I decided to limit the graph using frame numbers instead of neuron_ID for flexibility, so
you need to look in the data frame the desired start and end 

In [ ]:
#for neuron 31:
fig, axs = Bubblewrap.compare_runs([bw])

for line in photostim[:, 0]:
    for ax in axs[:,0]:
        ax.axvline(line, color='r')
